In [7]:

#code to just consume data from kafka consumer data format is
# output of consumer is json, after deserializer list of dicts
# pip install kafka-python

from kafka import KafkaConsumer
import json
import pandas as pd
# Initialize Kafka consumer
consumer = KafkaConsumer(
    'publictransportstream',  # Topic name
    bootstrap_servers="localhost:9092",   #'192.168.237.208:9092',
    auto_offset_reset='latest',
    enable_auto_commit=True,
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))
)

# Consume messages
print("Listening for messages...")
res_list=[]
for message in consumer:
    print(f"Raw message: {message.value}")
    




Listening for messages...
Raw message: {'Unnamed: 0': 145589, 'transition_date': '2020-02-01', 'transition_hour': 14, 'transport_type_id': 2, 'road_type': 'RAYLI', 'line': 'YENIKAPI - HACIOSMAN', 'transfer_type': 'Normal', 'number_of_passage': 12, 'number_of_passenger': 12, 'product_kind': 'UCRETSIZ', 'transaction_type_desc': 'Ucretsiz', 'town': 'FATIH', 'line_name': 'M2', 'station_poi_desc_cd': 'VEZNECILER KUZEY'}
Raw message: {'Unnamed: 0': 4714557, 'transition_date': '2020-02-15', 'transition_hour': 5, 'transport_type_id': 2, 'road_type': 'RAYLI', 'line': 'YENIKAPI - HACIOSMAN', 'transfer_type': 'Normal', 'number_of_passage': 10, 'number_of_passenger': 10, 'product_kind': 'INDIRIMLI1', 'transaction_type_desc': 'Indirimli Abonman', 'town': 'SISLI', 'line_name': 'M2', 'station_poi_desc_cd': 'ATATURK OTO SANAYI KUZEY'}
Raw message: {'Unnamed: 0': 8449382, 'transition_date': '2020-02-26', 'transition_hour': 9, 'transport_type_id': 2, 'road_type': 'RAYLI', 'line': 'YENIKAPI - HACIOSMAN',

KeyboardInterrupt: 

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ---------------------------------------- 0.0/782.3 kB ? eta -:--:--
     ------------- -------------------------- 262.1/782.3 kB ? eta -:--:--
     -------------------------------------- 782.3/782.3 kB 4.8 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached Cython-3.0.11-cp310-cp310-win_amd64.whl.metadata (3.2 kB)
Using cached Cython-3.0.11-cp310-cp310-win_amd64.whl (2.8 MB)
  Created wheel for happybase: filename=happybase-1.2.0-py2.py3-none-any.whl size=26614 sha256=efb0c1876a22d3723bdfdc381ffd613e6ee2ff6353e746bfed5804ef76db2d90
  Stored in directory: c:\users\faruk\appdata


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [10]:
#pip install happybase, library to connect hbase database
import happybase
conn = happybase.Connection(host='localhost', port=9089)
conn.create_table('test', {'cf': dict()})
print(conn.tables())

[b'test']


In [12]:
from kafka import KafkaConsumer
import json
import happybase

# Initialize Kafka consumer
consumer = KafkaConsumer(
    'publictransportstream',  # Topic name
    bootstrap_servers="localhost:9092",  # Adjust to your Kafka broker address
    auto_offset_reset='latest',
    enable_auto_commit=True,
    value_deserializer=lambda x: json.loads(x.decode('utf-8'))  # Deserialize JSON data
)

# Connect to HBase Thrift Server
connection = happybase.Connection(host='localhost')  # Replace 'localhost' with HBase server address
connection.open()

# Define the table name and column family
table_name = 'public_transport'
column_family = 'cf'

# Check if the table exists; create it if it doesn't
if table_name.encode() not in connection.tables():
    connection.create_table(
        table_name,
        {column_family: dict()}  # Define the column family
    )

# Access the HBase table
table = connection.table(table_name)

# Consume messages and write to HBase
print("Listening for messages...")
for message in consumer:
    data = message.value  # Deserialize the message
    print(f"Received: {data}")
    
    # Prepare data for HBase (ensure column family prefix)
    row_key = str(data['Unnamed: 0'])  # Use 'Unnamed: 0' as the unique row key
    hbase_data = {
        f"{column_family}:transition_date": data.get('transition_date', ''),
        f"{column_family}:transition_hour": str(data.get('transition_hour', '')),
        f"{column_family}:transport_type_id": str(data.get('transport_type_id', '')),
        f"{column_family}:road_type": data.get('road_type', ''),
        f"{column_family}:line": data.get('line', ''),
        f"{column_family}:transfer_type": data.get('transfer_type', ''),
        f"{column_family}:number_of_passage": str(data.get('number_of_passage', '')),
        f"{column_family}:number_of_passenger": str(data.get('number_of_passenger', '')),
        f"{column_family}:product_kind": data.get('product_kind', ''),
        f"{column_family}:transaction_type_desc": data.get('transaction_type_desc', ''),
        f"{column_family}:town": data.get('town', ''),
        f"{column_family}:line_name": data.get('line_name', ''),
        f"{column_family}:station_poi_desc_cd": str(data.get('station_poi_desc_cd', 'nan'))
    }
    
    # Write to HBase
    table.put(row_key, hbase_data)
    print(f"Data written to HBase with row key: {row_key}")

Listening for messages...
Received: {'Unnamed: 0': 145589, 'transition_date': '2020-02-01', 'transition_hour': 14, 'transport_type_id': 2, 'road_type': 'RAYLI', 'line': 'YENIKAPI - HACIOSMAN', 'transfer_type': 'Normal', 'number_of_passage': 12, 'number_of_passenger': 12, 'product_kind': 'UCRETSIZ', 'transaction_type_desc': 'Ucretsiz', 'town': 'FATIH', 'line_name': 'M2', 'station_poi_desc_cd': 'VEZNECILER KUZEY'}
Data written to HBase with row key: 145589
Received: {'Unnamed: 0': 4714557, 'transition_date': '2020-02-15', 'transition_hour': 5, 'transport_type_id': 2, 'road_type': 'RAYLI', 'line': 'YENIKAPI - HACIOSMAN', 'transfer_type': 'Normal', 'number_of_passage': 10, 'number_of_passenger': 10, 'product_kind': 'INDIRIMLI1', 'transaction_type_desc': 'Indirimli Abonman', 'town': 'SISLI', 'line_name': 'M2', 'station_poi_desc_cd': 'ATATURK OTO SANAYI KUZEY'}
Data written to HBase with row key: 4714557
Received: {'Unnamed: 0': 8449382, 'transition_date': '2020-02-26', 'transition_hour': 9,

KeyboardInterrupt: 